# Probability and Basic Statistics

### Descriptor: Skill Level
1. Able to select an appropriate probability analysis based on nature of the data and business knowledge.
2. Able to understand and apply advanced probability methods, such as Bayes Theorem, random number generation, central limit theorem, etc.
3. Able to choose the correct hypothesis testing and confidence interval approaches for a given business problem.
4. Able to define and perform estimation using techniques such as maximum likelihood estimation (MLE) and least squares.


### 1. Appropriate Probability Analysis

### 2. Advanced Probability Methods

### 3. Hypothesis Testing and Confidence Interval

### 4. Maximum Likelihood Estimation (MLE)